# IPIN 2025 Flowcean Hands-on Session

## Section 1 : Load and Prepare the Training Data

In [ ]:
import flowcean.cli

config = flowcean.cli.initialize()


### Task 1.1 Load Rosbags and Choose Inputs

In [ ]:
topics = {
    "/turtle1/cmd_vel": ["linear.x", "angular.z"],
    "/turtle1/pose": ["x", "y", "theta"],
}

### Task 1.2 Create Training Data Frame

In [ ]:
from _helper_functions import ShiftInTime
from flowcean.polars import DataFrame, ExplodeTimeSeries, ZeroOrderHold

transforms = (
    ZeroOrderHold(
        features=[
            "/turtle1/cmd_vel",
            "/turtle1/pose",
        ],
        name="measurements",
    )
    | ExplodeTimeSeries("measurements")
    | ShiftInTime(
        features=["/turtle1/pose/x", "/turtle1/pose/y", "/turtle1/pose/theta"],
        steps=1,
        suffix="_next",
    )
)

samples_train = (
    DataFrame.from_rosbag(config.rosbag.training_path, topics=topics) | transforms
)
samples_eval = (
    DataFrame.from_rosbag(config.rosbag.evaluation_path, topics=topics) | transforms
)

## Section 2 : Select Learners across Libraries 

In [ ]:
inputs = [
    "/turtle1/pose/x",
    "/turtle1/pose/y",
    "/turtle1/pose/theta",
    "/turtle1/cmd_vel/linear.x",
    "/turtle1/cmd_vel/angular.z",
]
outputs = [
    "/turtle1/pose/x_next",
    "/turtle1/pose/y_next",
    "/turtle1/pose/theta_next",
]

### Task 2.1 Learner configuration

In [ ]:
from flowcean.sklearn import RandomForestRegressorLearner, RegressionTree
from flowcean.torch import LightningLearner, MultilayerPerceptron
from flowcean.xgboost import XGBoostRegressorLearner

regression_tree = RegressionTree(**config.training.tree)

random_forest = RandomForestRegressorLearner(
    **config.training.forest,
)

mlp = LightningLearner(
    module=MultilayerPerceptron(
        learning_rate=config.training.mlp.learning_rate,
        input_size=len(inputs),
        output_size=len(outputs),
    ),
    batch_size=config.training.mlp.batch_size,
    max_epochs=config.training.mlp.max_epochs,
)

xgb = XGBoostRegressorLearner()

### Task 2.2 Prepare Sequential Learning

In [ ]:
learners = [
    regression_tree,
    random_forest,
    mlp,
    xgb,
]

## Section 3: Training of the Models

### Task 3.1 Create a Sequential Learning Loop

In [ ]:
from flowcean.core import learn_offline

models = []
for learner in learners:
    print(f"Training model: {learner.name}")
    model = learn_offline(
        samples_train,
        learner,
        inputs=inputs,
        outputs=outputs,
    )
    models.append(model)


## Section 4 : Evaluation and Model Comparison

### Task 4.1 Chose Metrics for Evaluation

In [ ]:
from custom_metrics.euclidean_distance import MeanEuclideanDistance
from flowcean.sklearn import MeanAbsoluteError, MeanSquaredError, R2Score

metrics = [
    MeanAbsoluteError(),
    MeanSquaredError(),
    R2Score(),
    MeanEuclideanDistance(
        features=["/turtle1/pose/x_next", "/turtle1/pose/y_next"],
    ),
]

### Task 4.2 Create an Evaluation Loop

In [ ]:
from flowcean.core import evaluate_offline

report = evaluate_offline(
    models,
    environment=samples_eval,
    metrics=metrics,
    inputs=inputs,
    outputs=outputs,
)
report.great_table()

###  Task 4.3 Select a Model and Visualization  

In [ ]:
from _helper_functions import plot_predictions_vs_ground_truth

best_model = models[3]
print(f"Best model: {best_model.name}")

# Plots are saved under plots/
plot_predictions_vs_ground_truth(
    environment=samples_eval,
    input_names=inputs,
    output_names=outputs,
    models=models,
)

# save model to disk
best_model.save("model.fml")

## Section 5 : Final Task

In [ ]:
from _helper_functions import surprise

surprise()